### Import libraries and create DataFrame from csv 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("RB_Data_2024.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,ATT,YDS,Y/A,LG,20+,TD,REC,TGT,YDS.1,Y/R,TD.1,G,FPTS,FPTS/G
0,1,Alvin Kamara (NO),104,428,4.1,17,0,6,28,36,238,8.5,1,6,136.6,22.8
1,2,Derrick Henry (BAL),119,704,5.9,87,8,8,6,8,49,8.2,1,6,135.3,22.6
2,3,Saquon Barkley (PHI),91,482,5.3,65,4,4,14,17,92,6.6,1,5,105.4,21.1
3,4,Kyren Williams (LAR),95,360,3.8,30,1,6,14,15,71,5.1,1,5,97.1,19.4
4,5,Chuba Hubbard (CAR),86,485,5.6,38,3,2,22,24,93,4.2,1,6,95.8,16.0
5,6,Breece Hall (NYJ),83,310,3.7,42,3,2,26,34,204,7.8,1,6,93.4,15.6
6,7,Bijan Robinson (ATL),82,380,4.6,28,1,3,21,23,161,7.7,0,6,93.1,15.5
7,8,Jordan Mason (SF),114,609,5.3,38,7,3,8,10,75,9.4,0,6,92.4,15.4
8,9,D'Andre Swift (CHI),91,325,3.6,36,2,3,19,23,193,10.2,0,6,90.8,15.1
9,10,David Montgomery (DET),75,351,4.7,21,1,6,10,10,94,9.4,0,5,90.5,18.1


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [2]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['Y/A', 'LG', 'Y/R', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,Y/A,LG,Y/R,G,FPTS,FPTS/G,ATT/game,YDS/game,20+/game,TD/game,REC/game,TGT/game,YDS.1/game,TD.1/game
0,1,Alvin Kamara (NO),4.1,17,8.5,6,136.6,22.8,17.3,71.3,0.0,1.0,4.7,6.0,39.7,0.2
1,2,Derrick Henry (BAL),5.9,87,8.2,6,135.3,22.6,19.8,117.3,1.3,1.3,1.0,1.3,8.2,0.2
2,3,Saquon Barkley (PHI),5.3,65,6.6,5,105.4,21.1,18.2,96.4,0.8,0.8,2.8,3.4,18.4,0.2
3,4,Kyren Williams (LAR),3.8,30,5.1,5,97.1,19.4,19.0,72.0,0.2,1.2,2.8,3.0,14.2,0.2
4,5,Chuba Hubbard (CAR),5.6,38,4.2,6,95.8,16.0,14.3,80.8,0.5,0.3,3.7,4.0,15.5,0.2
5,6,Breece Hall (NYJ),3.7,42,7.8,6,93.4,15.6,13.8,51.7,0.5,0.3,4.3,5.7,34.0,0.2
6,7,Bijan Robinson (ATL),4.6,28,7.7,6,93.1,15.5,13.7,63.3,0.2,0.5,3.5,3.8,26.8,0.0
7,8,Jordan Mason (SF),5.3,38,9.4,6,92.4,15.4,19.0,101.5,1.2,0.5,1.3,1.7,12.5,0.0
8,9,D'Andre Swift (CHI),3.6,36,10.2,6,90.8,15.1,15.2,54.2,0.3,0.5,3.2,3.8,32.2,0.0
9,10,David Montgomery (DET),4.7,21,9.4,5,90.5,18.1,15.0,70.2,0.2,1.2,2.0,2.0,18.8,0.0


### Calculate the correlations for the final stats across different conditions

In [3]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
Y/A,0.63,0.27,0.11,0.33,0.34
LG,0.75,0.59,0.43,0.35,0.53
Y/R,0.61,0.36,-0.16,-0.05,0.19
ATT/game,0.92,0.90,0.84,0.53,0.80
YDS/game,0.92,0.90,0.82,0.55,0.80
20+/game,0.69,0.66,0.56,0.33,0.56
TD/game,0.85,0.84,0.81,0.78,0.82
REC/game,0.84,0.81,0.45,0.23,0.58
TGT/game,0.84,0.81,0.47,0.22,0.59
YDS.1/game,0.80,0.76,0.37,0.13,0.52


### Assign the weights for the final stats

In [4]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
Y/A,0.34,0.11,0.07
LG,0.53,0.28,0.08
Y/R,0.19,0.04,0.06
ATT/game,0.80,0.64,0.14
YDS/game,0.80,0.64,0.14
20+/game,0.56,0.31,0.08
TD/game,0.82,0.67,0.14
REC/game,0.58,0.34,0.08
TGT/game,0.59,0.34,0.08
YDS.1/game,0.52,0.27,0.08


### Multiply the assigned weights to the final stats and calculate the score

In [5]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,Y/A_weighted,LG_weighted,ATT/game_weighted,YDS/game_weighted,20+/game_weighted,TD/game_weighted,REC/game_weighted,TGT/game_weighted,YDS.1/game_weighted
0,1,Alvin Kamara (NO),22.8,0.3,1.3,2.4,9.7,0.0,0.1,0.4,0.5,3.0
1,2,Derrick Henry (BAL),22.6,0.4,6.7,2.7,16.0,0.1,0.2,0.1,0.1,0.6
2,3,Saquon Barkley (PHI),21.1,0.4,5.0,2.5,13.1,0.1,0.1,0.2,0.3,1.4
3,4,Kyren Williams (LAR),19.4,0.3,2.3,2.6,9.8,0.0,0.2,0.2,0.2,1.1
4,5,Chuba Hubbard (CAR),16.0,0.4,2.9,1.9,11.0,0.0,0.0,0.3,0.3,1.2
5,6,Breece Hall (NYJ),15.6,0.2,3.2,1.9,7.1,0.0,0.0,0.3,0.5,2.6
6,7,Bijan Robinson (ATL),15.5,0.3,2.2,1.9,8.6,0.0,0.1,0.3,0.3,2.0
7,8,Jordan Mason (SF),15.4,0.4,2.9,2.6,13.8,0.1,0.1,0.1,0.1,0.9
8,9,D'Andre Swift (CHI),15.1,0.2,2.8,2.1,7.4,0.0,0.1,0.3,0.3,2.4
9,10,David Montgomery (DET),18.1,0.3,1.6,2.0,9.6,0.0,0.2,0.2,0.2,1.4


### Model training

In [6]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 0.703020344597826
Test Set MSE: 0.5494451566276148
Random Forest Test Set MSE: 0.2332491403094135


/tmp/ipykernel_2270/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [7]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("RB_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,Y/A,G,FPTS,FPTS/G,ATT/game,YDS/game,20+/game,TD/game,REC/game,TGT/game,YDS.1/game,TD.1/game
Rank,,,,,,,,,,,,,,,,
2,1,Derrick Henry (BAL),10.00,1,5.9,6,135.3,22.6,19.8,117.3,1.3,1.3,1.0,1.3,8.2,0.2
3,2,Saquon Barkley (PHI),9.12,1,5.3,5,105.4,21.1,18.2,96.4,0.8,0.8,2.8,3.4,18.4,0.2
29,3,Joe Mixon (HOU),8.99,26,5.5,3,62.0,20.7,17.3,95.3,0.7,0.7,2.7,3.7,24.7,0.3
1,4,Alvin Kamara (NO),8.84,-3,4.1,6,136.6,22.8,17.3,71.3,0.0,1.0,4.7,6.0,39.7,0.2
11,5,Kenneth Walker III (SEA),8.41,6,4.6,4,88.0,22.0,12.8,58.5,1.0,1.2,5.2,6.0,34.0,0.0
4,6,Kyren Williams (LAR),7.83,-2,3.8,5,97.1,19.4,19.0,72.0,0.2,1.2,2.8,3.0,14.2,0.2
20,7,Jonathan Taylor (IND),7.80,13,4.8,4,72.6,18.2,18.0,87.2,1.0,1.0,1.5,2.5,19.2,0.0
15,8,J.K. Dobbins (LAC),7.39,7,5.4,5,78.8,15.8,16.2,87.6,0.8,0.6,2.4,2.6,10.0,0.0
8,9,Jordan Mason (SF),7.33,-1,5.3,6,92.4,15.4,19.0,101.5,1.2,0.5,1.3,1.7,12.5,0.0
